#### CC Outils d'optimisation pour les sciences des données et de la décision
##### Version 1.3 - *C. W. Royer, janvier 2021.*
 
Liste des corrections postérieures au premier dépôt :

V 1.1 - Correction de la question 2 - On demande de montrer que $L_{\max} \ge L/d$.

V 1.2 - Correction dans la formule de $w^*$ en question 8 ($\tfrac{\lambda}{L}$), correction d'un *obj3* en *obj4* dans le dernier bloc d'affichage.

V 1.3 - Correction dans l'équation (3) de la question 6, on cherche bien à minimiser la norme de $\mathbf{w}-\mathbf{y}$. Ajout de la signification du paramètre *nblocs* dans le code **rcd_lasso**.

# Outils d'optimisation pour les sciences des données et de la décision - Contrôle continu

# Master 2 ID 2020-2021

## Date de rendu : 22 janvier 2021

## Modalités de rendu

- Ce notebook est à rendre **individuellement**. Votre rendu devra comporter vos nom et prénom(s). Les réponses aux différentes questions devront être incorporées au notebook, et la partie pratique devra pouvoir être exécutée par l'enseignant.

- Votre version du notebook est à envoyer par mail à l'adresse clement.royer@dauphine.psl.eu. La date limite est fixée au **22 janvier 2021**.

## Ressources utiles

- *La version la plus récente de ce notebook se trouve dans l'onglet* Fichiers *de l'équipe Teams associée au cours, ou sur [ce lien](https://www.lamsade.dauphine.fr/~croyer/ensdocs/ODD/ODD-CC.ipynb).*

- *Le polycopié de cours est disponible [ici](https://www.lamsade.dauphine.fr/~croyer/ensdocs/ODD/PolyODD.pdf).*

- *Ce notebook utilise LaTeX pour la partie mathématiques.* ***Le bloc ci-dessous devra être executé pour permettre d'éditer le notebook de la même manière qu'un document LaTeX (en permettant, notamment, aux équations d'être numérotées et référencées dans le même bloc).***

In [ ]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

- Des macros LaTeX sont définies ci-dessous (mais n'apparaissent plus une fois le bloc exécuté). N'hésitez pas à incorporer vos propres macros !

$$
\newcommand{\E}[1]{\operatorname{\mathbb{E}}\left[#1\right]}
\DeclareMathOperator*{\argmin}{\text{argmin}}
\def\R{{\mathbb{R}}}
\def\vz{{\mathbf{z}}}
\def\vy{{\mathbf{y}}}
\def\vx{{\mathbf{x}}}
\def\vw{{\mathbf{w}}}
\def\vv{{\mathbf{v}}}
\def\vu{{\mathbf{u}}}
\def\vq{{\mathbf{q}}}
\def\ve{{\mathbf{e}}}
\def\va{{\mathbf{a}}}
\def\mX{{\mathbf{X}}}
\def\mQ{{\mathbf{Q}}}
\def\calC{{\mathcal{C}}}
\def\setB{{\mathcal{B}}}
\newcommand{\T}{\mathrm{T}}
$$

### Notations

- Dans la suite, $d$ et $n$ désigneront toujours des entiers supérieurs ou égaux à 1.
- Pour tout vecteur $\vw \in \R^d$, le $i$-ème coefficient de $\vw$ sera noté $[\vw]_i$.

# Sujet du notebook : Algorithmes de descente par coordonnées

Les méthodes de descente par coordonnées (ou *coordinate descent* en anglais) sont une classe d'algorithmes qui a connu un regain d'intérêt avec l'arrivée des problèmes de traitement de données massives, pour lesquels il peut être trop coûteux d'agir sur toutes les variables simultanément. Si ces approches sont parmi les plus anciennes, elles ont été revisitées au cours des deux dernières décennies et sont toujours un sujet de recherche actif.

Le but de ce notebook est donc de servir d'introduction aux méthodes de descente par coordonnées, dans un contexte générique mais aussi adapté à l'optimisation *parcimonieuse* ou creuse, pour laquelle on cherche des solutions possédant le plus de coefficients nuls possible.

# Partie I - Bases des méthodes de descente par coordonnées

Pour introduire le principe des méthodes de descente par coordonnées, on considère un problème générique de la forme 
$$
    \min_{\vw \in \R^d} f(\vw),
$$
où la fonction $f$ est supposée convexe et de classe $\calC^1$.
 
Un algorithme de descente par coordonnées part d'un point $\vw_0 \in \R^d$; à chaque itération $k$, la méthode choisit un indice $i_k \in \{1,\dots,d\}$, une taille de pas $\alpha_k>0$, et effectue l'itération suivante :
\begin{equation}
\label{eq:dcit} 
    \vw_{k+1} := \vw_k - \alpha_k \nabla_{i_k} f(\vw_k) \ve_{i_k},
\end{equation}
où $\nabla_{i_k}$ est une notation qui désigne la dérivée partielle par rapport à la $i_k$-ème coordonnée, càd 
$\nabla_{i_k} f(\vw) = [\nabla f(\vw)]_{i_k}$ pour tout $\vw \in \R^d$, et $\ve_{i_k}$ désigne le $i_k$-ème vecteur de la base canonique de $\R^d$.

Le but intrinsèque d'une telle méthode est donc de ramener la résolution d'un problème en $d$ dimensions à celle d'une suite de problèmes en une dimension, en modifiant une coordonnée à la fois. Dans le cas général, il ne suffira pas de faire $d$ itérations pour obtenir la convergence, en revanche des propriétés telles que la séparabilité permettent de garantir la pertinence d'un tel procédé.

## I - A) Propriétés élémentaires

**Question 1** On peut réécrire l'itération \eqref{eq:dcit} comme $\vw_{k+1}:=\vw_k+c_k \ve_{i_k}$, où
\begin{equation}
\label{eq:dcitpb}
    c_k = \argmin_{c \in \R}\ q_k(c), \quad \mbox{avec} \quad q_k(c):=\nabla_{i_k} f(\vw_k) (c-[\vw_k]_{i_k}) 
    + \frac{1}{2\alpha_k}(c-[\vw_k]_{i_k})^2.
\end{equation}

**1-a)** Justifier que ce problème est fortement convexe, et donner la valeur explicite de $c_k$. 

**1-b)** À quel principe algorithmique vu en cours pouvez-vous relier la caractérisation \eqref{eq:dcitpb} ?

**Question 2** Une des hypothèses classiques d'analyse de ce genre de méthodes consiste à supposer que le gradient de $f$ est lipschitzien selon chaque coordonnée, c'est-à-dire qu'il existe $d$ réels positifs $L_1,\dots,L_d$ tels que
$$
    \forall i=1,\dots,d,\ \forall (\vw,\vv) \in (\R^d)^2, 
    \quad \left| \nabla_i f(\vw) - \nabla_i f(\vv) \right| 
    \le L_i \|\vw-\vv\|.
$$
On peut alors définir la *constante de Lipschitz par coordonnées* $L_{\max}:= \max_{1 \le i \le d} L_i$.
Justifier que si le gradient $\nabla f$ est $L$-lipschitzien, alors $L_{\max} \ge \frac{L}{d}$.

**Question 3** Par analogie avec les approches de type descente de gradient, pourquoi peut-on envisager le choix de taille de pas $\alpha_k = \frac{1}{L_{i_k}}$ à l'itération $k$ de l'algorithme ? Quel serait l'intérêt de choisir ce pas plutôt que $\frac{1}{L}$, comme vu en cours, ou que $\frac{1}{L_{\max}}$ ?
 

## *Réponses de la partie I - A)*

*Votre réponse à la question 1*

*1-a)*

*1-b)*

*Votre réponse à la question 2*

*Votre réponse à la question 3*

## I - B) Variantes aléatoires

On considère maintenant des variantes de la descente par coordonnées basée sur une sélection aléatoire des coordonnées.

Dans la suite de cette partie, on supposera que l'on a accès aux constantes de Lipschitz selon chaque coordonnée, notées $\{L_i\}_{i=1}^d$. On considère alors la variante de descente par coordonnée où un indice $i_k$ est tiré ***uniformément au hasard*** dans $\{1,\dots,d\}$, puis le nouvel itéré est calculé comme précédemment via l'itération 
$$
        \vw_{k+1} := \vw_k - \frac{\nabla_{i_k} f(\vw_k)}{L_{i_k}} \, \ve_{i_k},
$$
où l'on choisit $\frac{1}{L_{i_k}}$ comme taille de pas.


## Questions de la partie I - B)

**Question 4** Dans cette question, on suppose que la fonction $f$ (en plus d'être convexe) possède une unique solution $\vw^* \in \R^d$ et on note $f^* = f(\vw^*)$. Pour tout $\epsilon>0$, on peut alors montrer que l'algorithme de descente par coordonnées converge vers un itéré $\vw_k$ tel que 
$$
    \E{\vw_k} - f^* \le \epsilon 
$$
en au plus
$$
    \frac{2 d L_{\max} \|\vw_0-\vw^*\|^2}{\epsilon}
$$
itérations. Comparer ce résultat avec celui d'une descente de gradient sur le même problème en utilisant un pas $\frac{1}{L}$, avec $L$ une constante de Lipschitz pour $\nabla f$.


**Question 5** Il existe un algorithme qui combine la descente par coordonnées randomisée et l'accélération de 
Nesterov. Partant de $\mathbf{w}_0 \in \mathbb{R}^d$ et $\mathbf{v}_0=\mathbf{w}_0$, l'algorithme tire à chaque itération $k$ un indice $i_k$ uniformément au hasard entre $1$ et $d$, puis effectue la récursion :
\begin{equation}
\label{eq:nesterovcd}
    \left\{ 
        \begin{array}{lll}
            \vu_k &:= &\lambda_k \vv_k + (1-\lambda_k) \vw_k \\
            \vw_{k+1} &:= &\vu_k - \frac{1}{L_{i_k}} \nabla_{i_k} f(\vu_k) \ve_{i_k} \\
            \vv_{k+1} &:= &\mu_k \vv_k + (1-\mu_k) \vu_k - \frac{\gamma_k}{L_{i_k}}\nabla_{i_k} f(\vu) \ve_{i_k},
        \end{array}
    \right.
\end{equation}
où $\{\lambda_k,\mu_k,\gamma_k\}$ sont des suites de réels déterminés en fonction de $d$ (voire de la constante de convexité forte si $f$ est fortement convexe).

Cet algorithme obtient une meilleure complexité en $\mathcal{O}(\frac{1}{k^2})$. En observant l'itération \eqref{eq:nesterovcd}, quel inconvénient pratique décelez-vous dans ces récursions ?

**Question 6** Avec la stratégie "randomisée" décrite dans cette section, l'algorithme de descente par coordonnées rappelle l'algorithme du gradient stochastique. Il existe de fait une connection forte entre ces deux méthodes. Pour l'illustrer, on considère le problème aux moindres carrés linéaires
\begin{equation}
\label{eq:ermsg}
    \min_{\vv \in \R^d} \frac{1}{2d} \left\|\mQ \vv -\vy \right\|^2 = \frac{1}{2d} \sum_{i=1}^d (\vq_i^\T \vv - [\vy]_i)^2,
\end{equation}
où $\mQ = \left[ \begin{array}{c} \vq_1^\T \\ \vdots \\ \vq_d^\T \end{array} \right]$ est une matrice orthogonale, c'est-à-dire inversible avec $\mQ^\T = \mQ^{-1}$. Résoudre le problème \eqref{eq:ermsg} est alors équivalent à résoudre le problème
\begin{equation}
\label{eq:ermcd}
    \min_{\vw \in \R^d} \frac{1}{2} \left\|\vw -\vy \right\|^2 = \frac{1}{2} \sum_{i=1}^d ([\vw]_i - [\vy]_i)^2.
\end{equation}

Montrer qu'une itération du gradient stochastique appliqué à \eqref{eq:ermsg} et une itération de la descente par coordonnées randomisée appliquée à \eqref{eq:ermcd} avec la même taille de pas et le même tirage d'indice sont équivalentes, dans le sens où si $\vv_k = \mQ^\T \vw_k$, alors $\vv_{k+1}=\mQ^\T \vw_{k+1}$.

## *Réponses de la partie I - B)*

*Votre réponse à la question 4*

*Votre réponse à la question 5*

*Votre réponse à la question 6*


# PARTIE II - Descente par coordonnées et optimisation parcimonieuse

Dans cette partie, on étudie l'intérêt des méthodes de descente par coordonnées pour les problèmes de la forme
$$
    \min_{\vw \in \R^d} f(\vw) + \lambda \Omega(\vw),
$$
où $f$ sera une fonction de classe $\calC^1$, $\lambda>0$ et $\Omega$ est un terme de régularisation. On supposera que $\Omega$ possède une structure dite séparable, c'est-à-dire qu'il existe une fonction $h:[0,\infty) \rightarrow [0,\infty)$ telle que
$$
    \forall \vw \in \R^d, \quad \Omega(\vw) = \sum_{i=1}^d h(\left|[\vw]_i \right|).
$$
L'exemple classique d'une telle fonction est la fonction $h(t):=t$, qui conduit à une régularisation en norme $\ell_1$ de la forme $\Omega(\vw):=\|\vw\|_1$, avec $\|\vw\|_1 := \sum_{i=1}^d |[\vw]_i|$. On se concentrera sur ce choix dans ce notebook pour montrer l'intérêt des méthodes de descente par coordonnées, mais on notera que les idées majeures présentées ci-dessous s'étendent à d'autres régularisations de la forme pré-citée.


## II - A) Un problème unidimensionnel

On considère le problème
$$
    \min_{w \in \mathbb{R}}\ f_1(w), \quad \mbox{avec} \quad f_1(w):= a (w-u) +  \frac{L}{2} (w-u)^2 + \lambda |w|,
$$
où $a,u \in \mathbb{R}$, $L>0$, et $\lambda \ge 0$.

## Questions de la partie II - A)

**Question 7** Montrer que ce problème est fortement convexe.

**Question 8** On rappelle que la fonction $f_1$ est continue mais non dérivable partout. Comme vu en cours, son *sous-différentiel* est décrit par:
$$
    \partial f_1(w) := \left\{
        \begin{array}{ll}
            a + L(w-u) + \lambda &\mbox{si $w>0$}\\
            a + L(w-u) -\lambda &\mbox{si $w<0$} \\
            [a + L(w-u)-\lambda,a + L(w-u)+\lambda] &\mbox{si $w=0$.}
        \end{array}
    \right.
$$
Ainsi, le minimum global de $\min_{w \in \mathbb{R}} f_1(w)$, noté $w^*$, est caractérisé par la condition 
$$ 
    0 \in \partial f_1(w^*).
$$

En utilisant ce sous-différentiel, montrer que $w^*$ est donné par
$$
    w^* = \left\{
        \begin{array}{ll}
            u-\tfrac{a}{L} - \tfrac{\lambda}{L} &\mbox{si $u-\tfrac{a}{L} > \tfrac{\lambda}{L}$} \\
            u-\tfrac{a}{L} + \tfrac{\lambda}{L} &\mbox{si $u-\tfrac{a}{L} < -\tfrac{\lambda}{L}$} \\
            0 &\mbox{si $u-\tfrac{a}{L} \in [-\tfrac{\lambda}{L},\tfrac{\lambda}{L}]$.}
        \end{array}
    \right.
$$

## *Réponses de la partie II - A)*

*Votre réponse à la question 7*

*Votre réponse à la question 8*

## II-B) Cas général et descente de coordonnées par blocs

On considère maintenant un problème de type LASSO, aussi appelé problème de poursuite de base (de l'anglais *basis pursuit*) en traitement du signal. Pour un jeu de données formé par une matrice $\mX \in \R^{n \times d}$ et un vecteur $\vy \in \R^d$, ce problème s'écrit
\begin{equation}
\label{eq:basispursuit}
    \min_{\vw \in \R^d} f(\vw)+ \lambda \|\vw_1\|, \quad \mbox{avec} \quad
    f(\vw):= \frac{1}{2 n} \|\vy - \mX \vw \|^2.
\end{equation}
Il s'agit donc d'un problème avec régularisation $\ell_1$. Pour le résoudre, nous allons utiliser une approche par blocs de coordonnées, dans laquelle nous utiliserons un bloc $\setB_k \subset \{1,\dots,d\}$ de coordonnées tiré uniformément au hasard à chaque itération. Le nouveau point sera ainsi déterminé en résolvant le sous-problème
\begin{equation}
\label{eq:coord}
    \vw_{k+1} \in \argmin_{\vw} f_{\setB_k}(\vw):= f(\vw_k) + \sum_{i \in \setB_k} \left\{ 
    \left([\vw]_i-[\vw_k]_i\right) \nabla_i f(\vw_k)+ \frac{L_i}{2} \left([\vw]_i-[\vw_k]_i\right)^2 
    + \lambda \left| [\vw_i] \right| \right\},
\end{equation}
où $L_{i_k}$ représente la constante de Lipschitz correspondant à la $i_k$-ème coordonnée, que l'on suppose connue comme en partie I-B).

## Questions de la partie II - B)


**Question 9**  On cherche à implémenter l'algorithme ci-dessus.

**9-a)** En utilisant le résultat de la partie II-A), justifier que le nouvel itéré de l'algorithme de descente coordonnées par blocs est donné par
\begin{equation}
\label{eq:bcdup}
    \forall i \in \{1,\dots,d\}, \quad 
    [\vw_{k+1}]_i = \left\{
    \begin{array}{ll}
        [\vw_k]_i &\mbox{si $i \notin \setB_k$} \\
        [\vw_k]_i - \tfrac{1}{L_i} \nabla_i f(\vw_k) - \tfrac{\lambda}{L_i} &\mbox{si $i \in \setB_k$ et } 
        [\vw_k]_i - \tfrac{1}{L_i} \nabla_i f(\vw_k) > \tfrac{\lambda}{L_i} \\
        [\vw_k]_i - \tfrac{1}{L_i} \nabla_i f(\vw_k) + \tfrac{\lambda}{L_i} &\mbox{si $i \in \setB_k$ et } 
        [\vw_k]_i - \tfrac{1}{L_i} \nabla_i f(\vw_k) < - \tfrac{\lambda}{L_i} \\
        0 &\mbox{sinon.}
    \end{array}
    \right.
\end{equation}

**9-b)** Si on choisit tous les $L_i$ égaux, à quel algorithme vu en cours correspond la variante par blocs pour laquelle $|\setB| = n$ ?

**Question 10** Compléter le code-ci-dessous pour implémenter un algorithme de descente par coordonnées randomisé adapté au problème \eqref{eq:basispursuit} en utilisant la formule \eqref{eq:bcdup}. Tester ensuite plusieurs valeurs de blocs de coordonnées en utilisant le script fourni, et commenter vos résultats.


## *Réponses aux questions de la partie II - B)*

*Votre réponse à la question 9*

*Vos commentaires concernant la question 10*

In [ ]:
# Implémentation de l'algorithme de descente par coordonnées randomisé (RCD)
# À compléter pour répondre à la question 10
def rcd_lasso(w0,X,y,lbda,nblocs=1,nits=500): 
    """
        Code de la descente par coordonnées pour les problèmes de type LASSO de la forme
        
        Entrées :
            w0: Point initial
            X : Matrice du problème concerné
            y : Vecteur du problème concerné
            lbda : Coefficient de régularisation
            nblocs : Nombre de coordonnées considérées à chaque itération
            nits: Nombre maximum d'itérations à effectuer, utilisé comme critère d'arrêt
            
        Sorties :
            w_output : Dernier itéré calculé par l'algorithme
            objvals : Historique des valeurs de fonction (tableau Numpy de longueur au plus nits)
            nnzvals : Historique de parcimonie des itérés (tableau Numpy de longueur au plus nits)
    """
    
    ############
    # Initialisation des historiques
    objvals = []
    nnzvals = []
    
    # Valeur initiale de l'itéré (le .copy() permet de ne pas modifier le point de départ) 
    w = w0.copy()

    # Initialisation de l'indice d'itération
    k=0    
    
    # Dimensions
    n,d = X.shape
    
    # Constantes de Lipschitz
    ell = norm(np.matmul(X.T,X),axis=0)
    
    # Calcul de l'objectif en le point initial+Ajout à l'historique
    obj = norm(X.dot(w) - y) ** 2 / (2. * n) + lbda * norm(w,1)
    objvals.append(obj)
    
    # Calcul du nombre de coefficients non nuls dans l'itéré
    nnzvals.append(np.count_nonzero(w))
    
    #### PARTIE A COMPLETER
    # Calcul du gradient de la partie lisse en le point initial
    g = # A compléter
    #### FIN PARTIE A COMPLETER

    #########################
    # Début boucle principale
    while (k < nits):
        
        ### DEBUT PARTIE A COMPLETER
        # 
        # Tirage aléatoire de la ou des coordonnées
        
        # Calcul du nouvel itéré par composantes (modifier le vecteur w existant)
                
        # Calcul du gradient de la partie lisse en le nouveau point
        g = # A completer
             
        ### FIN PARTIE A COMPLETER

        # Calcul de l'objectif en le nouveau point ()
        obj = norm(X.dot(w) - y) ** 2 / (2. * n) + lbda * norm(w,1)
        objvals.append(obj)
        
        # Calcul du nombre de coefficients non nuls en le nouveau point
        nnzvals.append(np.count_nonzero(w))
        
        k += 1  
    # Fin boucle principale   
    #######################
    
    w_output = w.copy()
          
    return w_output, np.array(objvals), np.array(nnzvals)

In [ ]:
# Script de validation de l'implémentation de la question 10

################# Imports préliminaires
%matplotlib inline

import numpy as np # NumPy (calcul scientifique)
from scipy.linalg import norm # Norme euclidienne
import matplotlib.pyplot as plt # Affichage

############### Etape 1 - Génération des données avec une vérité terrain parcimonieuse
#
# Les coefficients de X suivent une loi normale N(0,1/n)
# Le vecteur y est obtenu via y = X*w+eps, où
#    w est un vecteur creux (10% de coefficients non nuls)
from numpy.random import multivariate_normal, randn

d = 200
n = 200
s = round(0.9*min(d,n))
X = multivariate_normal(np.zeros(d), (1/n)*np.identity(d), size=n)
idx = np.arange(d)

# Coefficients du vrai modèle ("vérité terrain/ground truth")
wtrue = (-1)**idx * np.exp(-idx / 10.)
ip = np.random.permutation(d)
wtrue[ip[0:s]]=0

Xw = X.dot(wtrue)
std = (0.01/n)*(norm(Xw)**2)
noise = std * randn(n)
y = Xw + noise

w0 = np.ones(d)
lbda = 1/ (n**0.5)


################# Etape 2 - Tester plusieurs valeurs de tailles de bloc

nb=1
nits=2000
w1,obj1,nnz1 = rcd_lasso(w0,X,y,lbda,nb,nits)
print('Nombre de bloc(s) :',nb)
print('Valeur objectif :',obj1[-1])
print('Pourcentage coefficients non nuls :',(d-nnz1[-1])*100/d)
nb=int(d/100)
w2,obj2,nnz2 = rcd_lasso(w0,X,y,lbda,nb,nits)
print('Nombre de bloc(s) :',nb)
print('Valeur objectif :',obj2[-1])
print('Pourcentage coefficients non nuls :',(d-nnz2[-1])*100/d)
nb=int(d/20)
w3,obj3,nnz3 = rcd_lasso(w0,X,y,lbda,nb,nits)
print('Nombre de bloc(s) :',nb)
print('Valeur objectif :',obj3[-1])
print('Pourcentage coefficients non nuls :',(d-nnz3[-1])*100/d)
nb=d
w4,obj4,nnz4 = rcd_lasso(w0,X,y,lbda,nb,nits)
print('Nombre de bloc(s) :',nb)
print('Valeur objectif :',obj4[-1])
print('Pourcentage coefficients non nuls :',(d-nnz4[-1])*100/d)

################# Etape 3 - Affichage des résultats



# En termes de fonction objectif (échelle logarithmique sur l'axe des ordonnées)
plt.figure(figsize=(7, 5))
plt.semilogy(obj1, label="nb=1", color='indianred',lw=2)
plt.semilogy(obj2, label="nb=d/100", color='brown', lw=2)
plt.semilogy(obj3, label="nb=d/20", color='red', lw=2)
plt.semilogy(obj4, label="nb=d", color='darkred', lw=2)
plt.title("Convergence des méthodes", fontsize=16)
plt.xlabel("#Iterations", fontsize=14)
plt.ylabel("Objectif (log)", fontsize=14)
plt.legend()

# Évolution du nombre de coordonnées nulles
plt.figure()
itnum = np.arange(nits+1)
print(itnum.shape)
print(nnz4.shape)
plt.scatter(itnum,nnz1,color='m',marker='o',label='nb=1')
plt.scatter(itnum,nnz2,color='indigo',marker='d',label='nb=d/100')
plt.scatter(itnum,nnz3,color='b',marker='*',label='nb=d/20')
plt.scatter(itnum,nnz4,color='dodgerblue',marker='x',label='nb=d')
plt.title("Parcimonie des itérés", fontsize=16)
plt.xlabel("#Iterations", fontsize=14)
plt.ylabel("Coefficients non nuls", fontsize=14)
plt.legend()